### Module and the `import` Statement

- Any Python source file can be used as a module.

- To load this code as a module, use the statement `import`. The first time `import` is used to load a module, it does three things:
    - It **creates a new namespace** that serves as a container for all the objects defined in the corresponding source file. This is **the namespace accessed when functions and methods defined within the module use the `global` statement**.
    - It **executes the code** contained in the module within the newly created namespace. If a module carries out a computation or produces output in addition to defining variables, functions, and classes, you will see the result.
    - It **creates a name within the caller that refers to the module namespace** (can be just the module's name if `as` is not used).
 
- To import multiple modules, you can supply import with a **comma-separated list of module names**.

- The name used to refer to a module can be changed using the **`as` qualifier**. 
    - When a module is loaded using a different name like this, the new name only applies to the source file or context where the import statement appeared. Other program modules can still load the module using its original name.
    - Changing the name of the imported module can be a useful tool for writing extensible code; see an example on Page 144 of 
[< Python Essential References >](https://www.evernote.com/shard/s191/nl/21353936/3a76bfd7-5b40-de76-dc58-c1805f99d416?title=Python%20Essential%20References).

- Modules are **first class objects** in Python.
    - This means that they can be assigned to variables, placed in data structures such as a list, and passed around in a program as a data.
    - Underneath the covers, a module object is a **layer over a dictionary** that is used to hold the contents of the module namespace. This dictionary is available as the `__dict__` of a module, and whenever you look up or change a value in a module, you’re working with this dictionary.
    
- The `import` statement can appear at any point in a program. However, the code in each module is **loaded and executed only once**, regardless of how often you use the `import` statement. Subsequent import statements **simply bind the module name to the
module object already created by the previous `import`**.

### Importing Selected Symbols from a Module

- The `from` statement is used to load specific definitions within a module into the current namespace. The `from` statement is identical to import except that **instead of creating a name referring to the newly created module namespace, it places references to one or more of the objects defined in the module into the current namespace**.

- The `from` statement also accepts a **comma-separated list** of object names. If you have a very long list of names to import, the **names can be enclosed in parentheses**. This makes it easier to break the import statement across multiple lines. In addition, the `as` qualifier can be used to **rename specific objects** imported with `from`.

- The `from module import *` statement may **only be used at the top level of a module**. In particular, it is **illegal to use this form of import inside function bodies** due to the way in which it interacts with function scoping rules. 
    - Modules can more precisely control the set of names imported by `from module import *` by defining the list `__all__`. 
    
- Importing definitions with the `from` form of import **does not change their scoping rules**. 
    - In particular, the global namespace for a function is **always the module in which the function was defined, not the namespace into which a function is imported and called**. This also applies to function calls.
    
- Another common confusion with the `from` form of import concerns the **behavior of global variables**.
    - It is not possible to use the from statement in a way that makes variables behave similarly as global variables or common blocks in languages such as C or Fortran.
    - If you want to have mutable global program parameters in your program, **put them in a module and use the module name explicitly using the `import` statement**.

### Execution as the Main Program

- Code might **execute as the main program or script**. This occurs when you supply the program as the script name to the interpreter.

- Each module defines a variable, `__name__`, that contains the module name. 
    - Programs can examine this variable to determine the module in which they’re executing. 
    - The top-level module of the interpreter is named `__main__`. **Programs specified on the command line or entered interactively run inside the `__main__` module**. This brings about the often-seen `if  __name__ == '__main__'`.

### The Module Search Path

- When loading modules, the interpreter searches the list of directories in `sys.path`. 
    - The first entry in `sys.path` is typically an empty string `''`, which refers to the **current working directory**. 
    - Other entries in sys.path may consist of directory names, `.zip` archive files. The **order in which entries are listed in `sys.path` determines the search order used when modules are loaded**.
    - To add new entries to the search path, simply add them to this list.

- Although the path usually contains directory names, `zip` archive files containing Python modules can also be added to the search path. This can be **a convenient way to package a collection of modules as a single file**. Specific locations within the directory structure of a zip file can also be used.

- Despite support for zip file imports, there are some restrictions to be aware of.
    - First, it is only possible import `.py`, `.pyw`, `.pyc`, and `.pyo` files from an archive.
    - Python **will not create `.pyc` and `.pyo` files when `.py` files are loaded from an archive** (described next). Thus, it is important to make sure these files are created in advance and placed in the archive in order to avoid poor performance when loading modules.

### Module Loading and Compilation

- When looking for a module (for example, `foo`), the interpreter searches each of the directories in `sys.path` for the following files (listed in search order):
    - A directory, `foo`, defining a package
    - `foo.pyd`, `foo.so`, `foomodule.so`, or `foomodule.dll` (compiled extensions)
    - `foo.pyo` (only if the `-O` or `-OO` option has been used)
    - `foo.pyc`
    - `foo.py` (on Windows, Python also checks for `.pyw` files.)

- For `.py` files, when a module is first imported, it’s **compiled into bytecode and written back to disk** as a `.pyc` file. On subsequent imports, the interpreter loads this precompiled bytecode **unless the modification date of the `.py` file is more recent (in which case, the `.pyc` file is regenerated)**.

- `.pyo` files are used in conjunction with the interpreter’s `-O` option. These files contain bytecode stripped of line numbers, assertions, and other debugging information. As a result, they’re somewhat smaller and allow the interpreter to run slightly faster.

- If the `-OO` option is specified instead of `-O`, **documentation strings are also stripped from the file**. This removal of documentation strings occurs **only when `.pyo` files are created—not when they’re loaded**.

- The automatic compilation of files into `.pyc` and `.pyo` files occurs only in conjunction with the `import` statement. **Programs specified on the command line or standard input don’t produce such files**. In addition, these files aren’t created if the directory containing a module’s `.py` file doesn't allow writing (e.g., either due to insufficient permission or if it’s part of a `zip` archive).The `-B` option to the interpreter also disables the generation of these files.

- If `.pyc` and `.pyo` files are available, it is not necessary for a corresponding `.py` file to exist. Thus, if you are packaging code and don’t wish to include source, you can merely bundle a set of `.pyc` files together. However, be aware that Python has extensive support for introspection and disassembly. Also, be aware that `.pyc` files tend to be version-specific.

- When import searches for files, it matches filenames in a **case-sensitive manner** — even on machines where the underlying file system is case-insensitive. As a general rule, you should avoid the use of module names that differ in case only.

### Module Reloading and Unloading

- Python **provides no real support for reloading or unloading of previously imported modules**. 
    - Although you can remove a module from `sys.modules`, this does not generally unload a module from memory. This is because references to the module object may still exist in other program components that used import to load that module.

- The fact that module references exist in many places makes it **generally impractical to reload a module after making changes to its implementation**. 

- Finally, it should be noted that `C/C++` extensions to Python cannot be safely unloaded or reloaded in any way.

So it may seem the **only resource is to restart the Python interpreter process**, after you have made changes to the code.

### Packages

- Packages allow a collection of modules to be grouped under a common package name. This technique **helps resolve namespace conflicts between module names used in different applications**.

- A package is **defined by creating a directory with the same name as the package and creating the file `__init__.py`** in that directory. You can then place additional source files, compiled extensions, and subpackages in this directory, as needed. 

- Different ways of importing submodules in a package.
    - `import Graphics.Primitive.fill`. This loads the submodule `Graphics.Primitive.fill`. The contents of this module have to be explicitly named, such as `Graphics.Primitive.fill.floodfill(img,x,y,color)`.
    - `from Graphics.Primitive import fill`. This loads the submodule fill but makes it available without the package prefix.
    - `from Graphics.Primitive.fill import floodfill`. This loads the submodule fill but makes the floodfill function directly accessible.

- Whenever any part of a package is first imported, the code in the file `__init__.py` is executed. Minimally, this file may be empty, but it can also contain code to perform package-specific initializations, such as `__all__` (see below). All the `__init__.py` files encountered during an import are executed.

- When a programmer does `from Graphics.Primitive import *`,  she usually wants to import all the submodules associated with a package into the current namespace. But in fact, this statement **just imports all the names that are defined in the `__init__.py` file** in the `Primitive` directory. 
    - The behavior can be modified by defining a list, `__all__`, that contains all the module names associated with the package. This list should be defined in the package `__init__.py` file.
    
- When submodules want to import other submodules within the same package, one can either **fully specify the name and path**, or use **a package relative import**: `from . import lines`.
    - The `.` used in the statement `from . import` lines refers to the same directory of the calling module.
    - In Python 3, import assumes an absolute path and will simply try to load module from the standard library. A relative import more clearly states your intentions.

- Some further notes
    - Relative imports **can only be specified** using the `from module import symbol` form of the import statement.
    - Also, `symbol` above has to be a **valid identifier**. So, a statement such as from `.. import Primitives.lines` is also illegal. 
    - Relative imports **can only be used within a package**; it is illegal to use a relative import to refer to modules that are simply located in a different directory on the filesystem.
    - Importing a package name alone doesn’t import all the submodules contained in the package. But since `__init__.py` is executed, **relative imports can be used to load all the submodules automatically in the `__init__.py`**; see an example on Page 151 of [< Python Essential References >](https://www.evernote.com/shard/s191/nl/21353936/3a76bfd7-5b40-de76-dc58-c1805f99d416?title=Python%20Essential%20References). 

- When Python imports a package, it defines a special variable, `__path__`, **which contains a list of directories that are searched when looking for package submodules (`__path__` is a package-specific version of the sys.path variable)**. 
    - `__path__` is accessible to the code contained in `__init__.py` files and initially contains a single item with the directory name of the package. 

## References
- [< Python Essential References >](https://www.evernote.com/shard/s191/nl/21353936/3a76bfd7-5b40-de76-dc58-c1805f99d416?title=Python%20Essential%20References), Chapter 8.